<a href="https://colab.research.google.com/github/YHONNEGO/MYMLProjectrepos/blob/main/advanced_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import os
cancerData=pd.read_csv('prostate.txt',delimiter='\t')

In [ ]:
trainCancer=cancerData[cancerData.loc[:,'train']=='T']
testCancer=cancerData[cancerData.loc[:,'train']=='F']
x_train=trainCancer.drop(columns=['id','lpsa','train'])
y_train=trainCancer.loc[:,'lpsa']
x_test= testCancer.drop(columns=['id','lpsa','train'])
y_test=testCancer.loc[:,'lpsa']

x_train_scaled=sklearn.preprocessing.scale(x_train, axis=0, with_mean=True,
with_std=True, copy=True)

x_test_scaled=sklearn.preprocessing.scale(x_test, axis=0, with_mean=True,
with_std=True, copy=True)

In [ ]:

# Function that gets prediction from matrix and Weights
def getPred(x,W):
    return(np.matmul(x,W))
# The basic loss, sum over the difference of y-ypred
def Loss(y,ypred):
    l=(y-ypred)**2
    return(l.sum())
#The Mean Square Error.
def MSE(X,Y,W):
    return((1/X.shape[0])*Loss(Y,getPred(X,W)))

In [ ]:
def GradDesc(X,Y,learnRate=0.01,epochs=2000,reg=0):

    global cacheLoss
    cacheLoss=[None]*epochs

   # Random Initialization of Weights
    Weights=np.random.rand(X.shape[1])

    Weights=np.array(Weights)
    Weights=Weights.reshape(-1,1)
    m=X.shape[0]

    for i in range(epochs):

        predictions=getPred(X,Weights)
        cacheLoss[i]=Loss(Y,predictions)

        Weights[0]=Weights[0]-(1/m)*learnRate*(np.matmul(X[:,0].transpose(),predictions-Y))

        for j in range(1,len(Weights)):

            Weights[j]=Weights[j]-(1/m)*learnRate*(np.matmul(X[:,j].transpose(),predictions-Y)+sum(np.dot(Weights[j],reg)))
    return(Weights)

In [ ]:
x_train_scaled=np.array(x_train_scaled)
y_train=np.array(y_train)
y_train=y_train.reshape(-1,1)
y_test=np.array(y_test)
y_test=y_test.reshape(-1,1)

#We need a column of ones to add as a bias term
addBias=np.ones([x_train_scaled.shape[0],1])
x_train_scaled=np.append(addBias,x_train_scaled,axis=1)

addBias=np.ones([x_test_scaled.shape[0],1])
x_test_scaled=np.append(addBias,x_test_scaled,axis=1)

# Get Linear Estimates

Wlinear=GradDesc(x_train_scaled,y_train)
Wlinear

array([[ 2.45234508],
       [ 0.70531873],
       [ 0.29164052],
       [-0.14189172],
       [ 0.21058105],
       [ 0.30777789],
       [-0.27753648],
       [-0.0111602 ],
       [ 0.26211981]])

In [ ]:
MSE(x_test_scaled,y_test,Wlinear)

0.5474832005044626

In [ ]:
X_train, X_Validate, Y_train, Y_Validate =  sklearn.model_selection.train_test_split(
x_train_scaled, y_train, test_size=0.33, random_state=42)

In [ ]:
def getRidgeLambda(x,y):

    bestMSE=10e100

    lamList=[l*0.05 for l in range(0,300)]

    global ridgeLambda

    for l in lamList:
        Wr=GradDesc(x,y,reg=l)
        if MSE(X_Validate,Y_Validate,Wr)< bestMSE:
            bestMSE=MSE(X_Validate,Y_Validate,Wr)
            ridgeLambda=l


    return(ridgeLambda)

ridgeLambda=getRidgeLambda(X_train,Y_train)
ridgeLambda

8.8

In [ ]:

Wridge=GradDesc(x_train_scaled,y_train,reg=ridgeLambda)
RidgeMSE=MSE(x_test_scaled,y_test,Wridge)

In [ ]:
Wridge

array([[ 2.45234508],
       [ 0.55245296],
       [ 0.27765753],
       [-0.09137754],
       [ 0.19282881],
       [ 0.26915467],
       [-0.102996  ],
       [ 0.02414358],
       [ 0.17775704]])

In [ ]:
RidgeMSE

0.5110920040462454

In [ ]:
def getLassoLambda(x,y):

    bestMSE=10e100

    alphaList=[l*0.1 for l in range(1,200)]

    for a in alphaList:

      lassoModel=sklearn.linear_model.Lasso(alpha=a,max_iter=5000,fit_intercept=False)

      lassoModel.fit(x,y)

      getPred=lassoModel.predict(X_Validate).reshape(-1,1)

      MSE=sum((Y_Validate-getPred)**2)
      if MSE < bestMSE:
            bestMSE=MSE
            lassoLambda=a
      return(lassoLambda)


lassoLambda=getLassoLambda(X_train,Y_train)
print(f'The ideal lambda for Lasso is {lassoLambda}')


The ideal lambda for Lasso is 0.1


In [ ]:

fitLasso=sklearn.linear_model.Lasso(alpha=lassoLambda,fit_intercept=False)

fitLasso.fit(x_train_scaled,y_train)

Wlasso=fitLasso.coef_

pz=fitLasso.predict(x_test_scaled).reshape(-1,1)

LassoMSE=(1/x_test_scaled.shape[0])*sum((y_test-pz)**2)

In [ ]:
Wlasso

array([ 2.35234509,  0.57054849,  0.22860076, -0.        ,  0.10505105,
        0.17102246,  0.        ,  0.        ,  0.06535352])

In [ ]:
LassoMSE

array([0.48223989])

In [ ]:
def getParametersElasticNet(x,y):
    bestMSE=10e100

    regList=[l*0.1 for l in range(1,500)]
    ratio=[i*0.001 for i in range(1,100)]

    global bestAlpha
    global bestRatio
    global bestElasticWeights

    for l1 in regList:
        for r in ratio:
            elasticModel=sklearn.linear_model.ElasticNet(alpha=l1,l1_ratio=r,fit_intercept=False,max_iter=3000,tol=1e-5)
            elasticModel.fit(x,y)
            getPred=elasticModel.predict(X_Validate).reshape(-1,1)

            MSE=sum((Y_Validate-getPred)**2)
            if MSE< bestMSE:
                bestMSE=MSE
                bestAlpha=l1
                bestRatio=r
                bestElasticWeights=elasticModel.coef_

    return(bestElasticWeights)



In [ ]:
elasticWeights=getParametersElasticNet(X_train,Y_train)


print(f'The ideal alpha for elastic net is {bestAlpha} and the best ratio is {bestRatio}')


fitElastic=sklearn.linear_model.ElasticNet(alpha=bestAlpha,l1_ratio=bestRatio,fit_intercept=False)
fitElastic.fit(x_train_scaled,y_train)
Welastic=fitElastic.coef_
pz=fitElastic.predict(x_test_scaled).reshape(-1,1)
ElasticMSE=(1/x_test_scaled.shape[0])*sum((y_test-pz)**2)
print(ElasticMSE)

ValueError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Load the Boston housing dataset
url = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
data = pd.read_csv(url)

# Split the data into predictors and target
X = data.drop('medv', axis=1)
y = data['medv']

# Split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Linear Regression model and fit it to the training data
model = LinearRegression()
model.fit(X_train, y_train)

# Use the model to make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the R-squared score
r2 = r2_score(y_test, y_pred)

# Calculate the adjusted R-squared score
n = X_test.shape[0]  # number of observations
k = X_test.shape[1]  # number of predictors

adj_r2 = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

print(f'R-squared: {r2}')
print(f'Adjusted R-squared: {adj_r2}')

R-squared: 0.6687594935356326
Adjusted R-squared: 0.619826236898851


In [ ]:
AIC<BIC